# Tropause Analysis Demonstration

This notebook presents a typical science use case. We will take advantage of the 
database utility available in awsgnssroutils.py, download radio occultation (RO) 
data files from the AWS Registry of Open Data respository of RO data, execute 
some basic numerical analysis, and plot results using matplotlib. 

## Imports and defaults

First, import Python modules. 

In [ ]:
import os
import json
import numpy as np
from scipy.optimize import fmin
from scipy.interpolate import interp1d
from netCDF4 import Dataset
from awsgnssroutils import RODatabaseClient
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator 

Plot default settings. 

In [ ]:
axeslinewidth = 0.5 
plt.rcParams.update( {
  'font.family': "Times New Roman", 
  'font.size': 8,  
  'font.weight': "normal", 
  'text.usetex': True, 
  'xtick.major.width': axeslinewidth, 
  'xtick.minor.width': axeslinewidth, 
  'ytick.major.width': axeslinewidth, 
  'ytick.minor.width': axeslinewidth, 
  'axes.linewidth': axeslinewidth } ) 

Define the tropical and extra-tropical tropopause bounds. The height 
bounds have units of gpm, or geopotential meters. Convert these numbers 
to geopotential energy per unit mass by multiplying by WMO standard gravity, 
which has units of J/kg/gpm. 

In [ ]:
tropical_tropopause = { 'height_bounds': np.array( [15.0e3,20.0e3] ) }
extratropical_tropopause = { 'height_bounds': np.array( [8.0e3,14.0e3] ) }
gravity = 9.80665

Create an RO database object to be used for database queries and downloading 
RO data. 

In [ ]:
HOME = os.path.expanduser( "~" )
databasepath = os.path.join( HOME, "local/rodatabase" )
db = RODatabaseClient( repository=databasepath, update=False )

## Select RO dataset

Select a subset of RO dataset to analyze for analysis using the 
database object. 

In [ ]:
datetimerange = ( "2020-01-01", "2020-01-02" )
occs = db.query( datetimerange=datetimerange )

At present, this gets metadata on one day of RO data, all RO missions. 
The bounds in datatimerange are strings of ISO-format times. Next, 
diagnostics on the occultations available. 

In [ ]:
print( f"Number of soundings found = {occs.size}" )
missions = occs.info( "mission" )
for mission in missions: 
    suboccs = occs.filter( missions=mission )
    print( f"Number of {mission} soundings found = {suboccs.size}" )
filetypes_dict = occs.info( "filetype" )
for filetype, number in filetypes_dict.items(): 
    center, AWStype = filetype.split("_")
    if AWStype == "atmosphericRetrieval": 
        print( f"Number of {filetype} data files = {number}" )

## Download data and analyze

Work with contributions from UCAR. Let's download the UCAR atmosphericRetrieval 
format files. The atmosphericRetrieval files contain RO retreivals of 
standard atmospheric variables: temperature (not "dry temperature"), pressure 
(not "dry pressure"), refractivity, partial pressure of water vapor, all as 
functions of geopotential. 

This notebook downloads data into the path "~/Data/rodata/". Download data by mission. 

In [ ]:
datadir = os.path.join( HOME, "Data/rodata" )
missions = occs.info( "mission" )
datafiles = {}
for mission in missions: 
    suboccs = occs.filter( missions=mission )
    files = suboccs.download( "ucar_atmosphericRetrieval", datadir, keep_aws_structure=True )
    datafiles.update( { mission: files } )

Analyze data by mission. Keep track of tropopause height, temperature, and pressure. 

In [ ]:
#  Initialize output variables. 

print( "Initializing data dictionaries.")
for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
    for mission in missions: 
        tropopause.update( { mission: { 'height': [], 'temperature': [], 'pressure': [], 'latitude': [] } } )

#  Loop over mission. 

for mission in missions: 
    print( f"Analyzing {mission} for tropopauses." )
    for file in datafiles[mission]: 
        
        #  Read in temperature and pressure profiles. 
        
        d = Dataset( file, 'r' )
        temperature = d.variables['temperature'][:]
        pressure = d.variables['pressure'][:]
        height = d.variables['geopotential'][:] / gravity
        latitude = d.variables['refLatitude'].getValue()
        d.close()
        
        for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
            rec = tropopause[mission]
            bounds = tropopause['height_bounds']
            
            i = np.argwhere( np.logical_and( height > bounds[0], height < bounds[1] ) ).squeeze()
            ftemperature = interp1d( height[i], temperature[i], kind="cubic", 
                                        bounds_error=False, fill_value=1.0e10 )
            j = temperature[i].argsort()[0]
            x0 = height[i[j]]
            height_min = fmin( ftemperature, x0=x0 )
            
            found = ( height_min > bounds[0]+100 and height_min < bounds[1]-100 )
            
            if found: 
                temperature_min = ftemperature( height_min )
                flnpressure = interp1d( height[i], np.log( pressure[i] ), kind="cubic", 
                                       bounds_error=False, fill_value=1.0e10 )
                pressure_min = np.exp( flnpressure(height_min) )
                rec['height'].append( height_min )
                rec['temperature'].append( temperature_min )
                rec['pressure'].append( pressure_min )
                rec['latitude'].append( latitude )

#  Convert to ndarrays. 

for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
    for mission in missions: 
        rec = tropopause[mission]
        for variable in [ 'height', 'temperature', 'pressure', 'latitude' ]: 
            rec[variable] = np.array( rec[variable] )


## Plot results

First, define latitude labels and mission colors. 


In [ ]:
tickvalues = np.arange( -90, 90.1, 30 )
ticklabels = []
for tickvalue in tickvalues: 
    if tickvalue > 0: 
        ticklabels.append( "{:}$^\circ$N".format( int(np.abs(tickvalue)) ) )
    elif tickvalue < 0: 
        ticklabels.append( "{:}$^\circ$S".format( int(np.abs(tickvalue)) ) )
    else: 
        ticklabels.append( "Eq" )

cmap = plt.get_cmap( "gist_ncar" )
nmissions = len( missions )
colors = { mission: cmap((imission+0.5)/nmissions) for imission, mission in enumerate(missions) }

Now, plot tropopause height. 

In [ ]:
plt.clf()

fig = plt.figure()
ax = fig.add_axes( [0.1,0.1,0.87,0.87] )

ax.set_xticks( tickvalues )
ax.xaxis.set_minor_locator( MultipleLocator(10) )
ax.set_xticklabels( ticklabels )
ax.set_yticks( np.arange(8,20.1,2) )
ax.set_ylabel( "Tropopause height [km]" )
ax.yaxis.set_minor_locator( MultipleLocator(0.5) )
for mission in missions: 
    first = True
    for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
        rec = tropopause[mission]
        args = { 's': 0.2, 'color': colors[mission] }
        if first: 
            args.update( { 'label': mission } )
        ax.scatter( rec['latitude'], rec['height']/1000, **args )
        first = False

ax.legend( loc="lower right" )
plt.show()

Now plot tropopause temperature. 

In [ ]:
plt.clf()

fig = plt.figure()
ax = fig.add_axes( [0.1,0.1,0.87,0.87] )

ax.set_xticks( tickvalues )
ax.xaxis.set_minor_locator( MultipleLocator(10) )
ax.set_yticks( np.arange(180,240.1,20) )
ax.yaxis.set_minor_locator( MultipleLocator(5) )
ax.set_ylim( 180, 240 )
ax.set_ylabel( "Tropopause temperature [K]" )

for mission in missions: 
    first = True
    for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
        rec = tropopause[mission]
        args = { 's': 0.2, 'color': colors[mission] }
        if first: 
            args.update( { 'label': mission } )
        ax.scatter( rec['latitude'], rec['temperature'], **args )
        first = False

ax.legend( loc="lower right" )
plt.show()

Tropopause pressure. 

In [ ]:
plt.clf()

fig = plt.figure()
ax = fig.add_axes( [0.1,0.1,0.87,0.87] )

ax.set_xticks( tickvalues )
ax.xaxis.set_minor_locator( MultipleLocator(10) )
ax.set_yticks( np.arange(300,49.9,-50) )
ax.yaxis.set_minor_locator( MultipleLocator(10) )
ax.set_ylim( 300, 30 )
ax.set_ylabel( "Tropopause pressure [hPa]" )

for mission in missions: 
    first = True
    for tropopause in [ tropical_tropopause, extratropical_tropopause ]: 
        rec = tropopause[mission]
        args = { 's': 0.2, 'color': colors[mission] }
        if first: 
            args.update( { 'label': mission } )
        ax.scatter( rec['latitude'], rec['pressure']/100, **args )
        first = False

ax.legend( loc="lower right" )
plt.show()